In [1]:
import torch
from model import MultiSeqBase
model = MultiSeqBase(n_labels=3, hidden_size=32, embedding_dim=4, dropout_p=0.2)
optim = torch.optim.Adam(model.parameters(), weight_decay=1e-3)

# Load model and optimizer
checkpoint = torch.load('test_model_and_optimizer.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])

In [2]:
from utils import db2Rseq, db2Hseq, dt2T, db2S, y2db

r_seq = db2Rseq()
h_seq = db2Hseq()
t = dt2T()
s_attrs, s_embed = db2S()

output = model(r_seq.float(), h_seq.float(), t.int(), s_attrs.float()).argmax(dim=1)

In [3]:
# get station id from database
from utils import db_connect, db2Sid

sid = db2Sid()

In [16]:
def y2db(outputs):
    sid = db2Sid()
    update_query = "UPDATE occupancy SET Occupancy_20 = %s WHERE Sid = %s"
    for values in list(zip(np.array(output), sid)):
        
        with connection.cursor() as cursor:
            cursor.execute(update_query, values)
        
    connection.commit()
    connection.close()

In [18]:
connection = db_connect()
connection.close()